In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [17]:
pip install sentence-transformers

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 1.6 MB/s eta 0:00:00
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/275.9 kB ? eta -:--:--
   -------------------- ------------------- 143.4/275.9 kB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 275.9/275.9 kB 3.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/204.2 MB ? eta -:--:--
   ---------------------------------------- 0.2/204.2 MB 3.9 MB/s eta 0:00:53
   ---------------------------------------- 0.4/204.2 MB 5.2 MB/s eta 0:00:40
   ---------------------------------------- 0.5/204.2 MB 4.7 MB/s eta 0:00:43
   ---------------------------------------- 0.6/204.2


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

import pandas as pd

books = pd.read_csv('books_cleaned.csv')

books['tagged_desc'].to_csv('tagged_desc.txt', sep=' ', index=False, header=False)

raw_descriptions = TextLoader("tagged_desc.txt", encoding="utf-8").load()

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50, separator='\n')

documents = text_splitter.split_documents(raw_descriptions)

embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

db_books = Chroma.from_documents(documents, embedding=embedding_function)


Created a chunk of size 1170, which is longer than the specified 500
Created a chunk of size 1216, which is longer than the specified 500
Created a chunk of size 962, which is longer than the specified 500
Created a chunk of size 845, which is longer than the specified 500
Created a chunk of size 881, which is longer than the specified 500
Created a chunk of size 1090, which is longer than the specified 500
Created a chunk of size 1191, which is longer than the specified 500
Created a chunk of size 515, which is longer than the specified 500
Created a chunk of size 754, which is longer than the specified 500
Created a chunk of size 730, which is longer than the specified 500
Created a chunk of size 723, which is longer than the specified 500
Created a chunk of size 1267, which is longer than the specified 500
Created a chunk of size 683, which is longer than the specified 500
Created a chunk of size 555, which is longer than the specified 500
Created a chunk of size 523, which is longe

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\karan\PycharmProjects\Book-recommender\.venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\karan\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k = 10)
docs

[Document(id='7b5e03dd-725e-4787-9075-cb29933b18c8', metadata={'source': 'tagged_desc.txt'}, page_content='"9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience."'),
 Document(id='86b21a6a-7160-4e62-b643-a2bc157ba54d', metadata={'source': 'tagged_desc.txt'}, page_content='"9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child\'s drawings and abstract art to playful photos. A brief text accompanies each image, introducing the baby to some basic -- and sometimes playful -- information about the subjects."'),
 Document(id='452f3ed8-9bce-4202-8c31-f14360b29b19', metadata={'source': 'tagged_de

In [24]:
isbn_str = docs[0].page_content.split()[0].strip().strip('"')

try:
    isbn_num = int(isbn_str)
    result = books[books["isbn13"] == isbn_num]
    print(result)
except ValueError as e:
    print(f"Error: {e}, ISBN value: {isbn_str}")


             isbn13      isbn10                                title subtitle  \
3903  9780786808069  0786808063  Baby Einstein: Neighborhood Animals      NaN   

                                authors        categories  \
3903  Marilyn Singer;Julie Aigner-Clark  Juvenile Fiction   

                                              thumbnail  \
3903  http://books.google.com/books/content?id=X9a4P...   

                                            description  published_year  \
3903  Children will discover the exciting world of t...          2001.0   

      average_rating  num_pages  ratings_count  missing_desc  age_of_the_book  \
3903            3.89       16.0          180.0             0             24.0   

      words_in_desc                   title_and_subtitle  \
3903             40  Baby Einstein: Neighborhood Animals   

                                            tagged_desc  
3903  9780786808069 Children will discover the excit...  


In [25]:
isbn_str = docs[0].page_content.split()[0].strip()
print(f"Extracted ISBN: {isbn_str}")  # Check if there are unwanted characters
books[books["isbn13"] == int(isbn_str.strip('"'))]


Extracted ISBN: "9780786808069


,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missing_desc,age_of_the_book,words_in_desc,title_and_subtitle,tagged_desc
3903,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,NaN,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,0,24.0,40,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...


In [26]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = 50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)]

In [30]:
retrieve_semantic_recommendations("sex")

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missing_desc,age_of_the_book,words_in_desc,title_and_subtitle,tagged_desc
26,9780006514855,0006514855,Girls' Night in,NaN,Jessica Adams;Chris Manby;Fiona Walker,American fiction,http://books.google.com/books/content?id=xLwHH...,'Girls' Night In' features stories about growi...,2000.0,3.26,586.0,140.0,0,25.0,45,Girls' Night in,9780006514855 'Girls' Night In' features stori...
172,9780060589288,0060589280,Eleven Minutes,A Novel,Paulo Coelho,Fiction,http://books.google.com/books/content?id=Qp0EZ...,"Eleven Minutes is the story of Maria, a young ...",2005.0,3.69,273.0,114826.0,0,20.0,151,Eleven Minutes: A Novel,9780060589288 Eleven Minutes is the story of M...
181,9780060598808,0060598808,Thanksgiving,NaN,Janet Evanovich,Fiction,http://books.google.com/books/content?id=O-t1G...,When Megan Murphy discovered a floppy-eared ra...,2006.0,3.60,228.0,10334.0,0,19.0,49,Thanksgiving,9780060598808 When Megan Murphy discovered a f...
214,9780060753634,0060753633,Mating in Captivity,Reconciling the Erotic and the Domestic,Esther Perel,Psychology,http://books.google.com/books/content?id=-HIhM...,A guide for loving couples who are looking to ...,2006.0,4.13,272.0,7699.0,0,19.0,50,Mating in Captivity: Reconciling the Erotic an...,9780060753634 A guide for loving couples who a...
261,9780060846732,0060846739,Sex with the Queen,"900 Years of Vile Kings, Virile Lovers, and Pa...",Eleanor Herman,History,http://books.google.com/books/content?id=5GSgQ...,In this follow-up to her bestselling Sex with ...,2006.0,3.91,336.0,3676.0,0,19.0,317,"Sex with the Queen: 900 Years of Vile Kings, V...",9780060846732 In this follow-up to her bestsel...
268,9780060872984,0060872985,Next,NaN,Michael Crichton,Fiction,http://books.google.com/books/content?id=ss1hX...,Is a loved one missing some body parts? Are bl...,2006.0,3.49,431.0,57257.0,0,19.0,207,Next,9780060872984 Is a loved one missing some body...
272,9780060884055,0060884053,How to Make Money Like a Porn Star,NaN,Neil Strauss;Bernard Chang,Comics & Graphic Novels,http://books.google.com/books/content?id=y7XAO...,Claudia Corvette. From her tousled bedroom hai...,2006.0,3.31,128.0,159.0,0,19.0,202,How to Make Money Like a Porn Star,9780060884055 Claudia Corvette. From her tousl...
288,9780060916466,006091646X,The Dance of Intimacy,A Woman's Guide to Courageous Acts of Change i...,Harriet Lerner,Psychology,http://books.google.com/books/content?id=tTKc8...,"In The Dance of Intimacy, the bestselling auth...",1990.0,4.06,255.0,7128.0,0,35.0,98,The Dance of Intimacy: A Woman's Guide to Cour...,"9780060916466 In The Dance of Intimacy, the be..."
359,9780061054907,0061054909,Love in Vein,NaN,Poppy Z. Brite,Fiction,http://books.google.com/books/content?id=AO_bZ...,A sexy new repackaging of the extremely popula...,1995.0,3.60,397.0,2391.0,0,30.0,103,Love in Vein,9780061054907 A sexy new repackaging of the ex...
483,9780099389019,0099389010,The Professor of Desire,NaN,Philip Roth,NaN,http://books.google.com/books/content?id=fNlzi...,"As a student in college, David Kepesh styles h...",1995.0,3.65,272.0,1950.0,0,30.0,98,The Professor of Desire,"9780099389019 As a student in college, David K..."


In [ ]:
from tqdm import tqdm

actu